In [ ]:
// to simulate Singtleton process, create single instance of MemoryCache
// this method is bad example because not deal with multiple thread process

Main();

void Main()
{
	MemoryCache.Create();
	MemoryCache.Create();
	MemoryCache.Create();
	MemoryCache.Create();
	MemoryCache.Create();
}

public class MemoryCache
{
	private static MemoryCache _instance;
	
	private MemoryCache() {
		Console.WriteLine("Created Singleton instance");
	}
	
	public static MemoryCache Create() {
		return _instance ?? (_instance = new MemoryCache());
	}	
}

In [ ]:
// Problem: create singleton instance, not support in multiple process
// this is to simulate above example in multiple thread, it will create multiple instance
// this is not what we want

Main();

void Main()
{
	int size = 8;
	Task[] tasks = new Task[size];
	for (int i = 0; i < size; i++){
		tasks[i] = Task.Run(() => MemoryCache.Create());
	}
	Task.WaitAll(tasks);
	//MemoryCache.Create();
}

public class MemoryCache
{
	private static int i = 0;
	private static MemoryCache _instance;

	private MemoryCache()
	{
		Console.WriteLine($"Created {i++}");
	}
	
	public static MemoryCache Create() {
		return _instance ?? (_instance = new MemoryCache());
	}
}

In [ ]:
// First Solution: static constructor
// when application start, the static constructor be called, if many many static constructor in application, it will take much time to start application

Main();
void Main()
{
	int size = 8;
	Task[] tasks = new Task[size];
	for (int i = 0; i < size; i++){
		tasks[i] = Task.Run(() => MemoryCache.Create());
	}
	Task.WaitAll(tasks);
	//MemoryCache.Create();
}


public class MemoryCache
{
	private static int i = 0;
	private static MemoryCache _instance;
	
	static MemoryCache(){
		_instance = new MemoryCache();
	}

	private MemoryCache()
	{
		Console.WriteLine($"Created {i++}");
	}
	
	public static MemoryCache Create() {
		return _instance;
	}
}

In [ ]:
// Solution: lock resource
// this is solution for above singleton in multiple thread
// add lock to instance, it wil always create one single instance

Main();
void Main()
{
	int size = 8;
	Task[] tasks = new Task[size];
	for (int i = 0; i < size; i++)
	{
		tasks[i] = Task.Run(() => MemoryCache.Create());
	}
	Task.WaitAll(tasks);
	//MemoryCache.Create();
}

public class MemoryCache
{
	private static int i = 0;
	private static MemoryCache _instance;
	private static object _cacheLock = new object();

	private MemoryCache()
	{
		Console.WriteLine($"Created {i++}");
	}

	public static MemoryCache Create()
	{
		// if _instance is null, continue for creation process
		if (_instance == null)
		{
			// lock the object, memory allows only one thread to access the object at one time
			lock (_cacheLock)
			{
				// after relase lock, do check again, to avoid duplicate creation
				if (_instance == null)
				{
					return _instance = new MemoryCache();
				}
			}
		}

		return _instance;
	}
}

In [ ]:
// Problem: access the Singleton cache
// this example use Contains method to check whether key exists, but it not work under multiple thread

Main();

void Main()
{
	int size = 10;
	Task[] tasks = new Task[size];
	for (int i = 0; i < size; i++)
	{
		tasks[i] = Task.Run(() =>
		{
			var c = MemoryCache.Create();
			if (!c.Contains("job_id", "job1"))
			{
				Console.WriteLine($"Big Operation");
			}
		});
	}
	Task.WaitAll(tasks);
}


public class MemoryCache
{
	private static MemoryCache cache;
	private static object cacheLock = new object();

	private readonly Dictionary<string, string> _registry;

	private MemoryCache() => _registry = new Dictionary<string, string>();

	public static MemoryCache Create()
	{
		if (cache == null)
		{
			lock (cacheLock)
			{
				if (cache == null)
				{
					return cache = new MemoryCache();
				}
			}
		}

		return cache;
	}

	public bool Contains(string key, string value)
	{
		return _registry.Contains(KeyValuePair.Create(key, value));
	}
}

In [ ]:
// Solution: use lock, to void multiple thread perform creation in same time

Main();

void Main()
{
	int size = 10;
	Task[] tasks = new Task[size];
	for (int i = 0; i < size; i++)
	{
		tasks[i] = Task.Run(() =>
		{
			var c = MemoryCache.Create();
			if (c.AquireKey("job_id", "job1"))
			{
				Console.WriteLine("Big Operation");
			}
		});
	}
	Task.WaitAll(tasks);
}


public class MemoryCache
{
	private static MemoryCache cache;
	private static object cacheLock = new object();

	private readonly Dictionary<string, string> _registry;

	private MemoryCache() => _registry = new Dictionary<string, string>();

	public static MemoryCache Create()
	{
		if (cache == null)
		{
			lock (cacheLock)
			{
				if (cache == null)
				{
					return cache = new MemoryCache();
				}
			}
		}

		return cache;
	}

	public bool Contains(string key, string value)
	{
		return _registry.Contains(KeyValuePair.Create(key, value));
	}

	public void Write(string key, string value)
	{
		_registry[key] = value;
	}


	public bool AquireKey(string key, string value)
	{
		lock (cacheLock)
		{
			if (Contains("job_id", "job1"))
			{
				return false;
			}
			Write("job_id", "job1");
			
			return true;
		}
	}
}

Big Operation
